In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
rc('font', **{ "size":13,}) #**{,,
rc('text', usetex=True)
import numpy.random as rnd
rnd.seed()
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import scipy.linalg as la
import pickle as cp
from scipy.special import gamma
from scipy.optimize import curve_fit as fit
from scipy.optimize import brentq
import scipy.fft as fft

l10 = lambda x: np.log(x) / np.log(10)

In [ ]:
def getK(N=200, alpha = 1, delta = .2, SR = False, **kwargs):
    xmin = 1
    endpoints = np.hstack([[0, xmin], np.exp(np.arange(1,N+1) * delta) * xmin ])
    ls= endpoints[:-1]
    rs = endpoints[1:]
    mid = (endpoints[1:] + endpoints[:-1]) / 2
    K = np.array([(abs(1/np.abs(x - rs) ** alpha - 1/np.abs(x-ls) ** alpha) 
        + abs(1/np.abs(x + rs) ** alpha - 1/np.abs(x  + ls) ** alpha)) for x in mid[1:]]) / alpha
    K[np.arange(N), np.arange(N)+1]=0
    K0 = np.sum(K, axis = 1)
    return K, K0
ba = lambda a: 2 ** (d-a) * np.pi**(d/2) * gamma((d-a)/2)/gamma(a/2)
def geteta(alpha):
    Dalpha = lambda eta: ba(eta) * ba(d-eta-alpha) / ba(alpha) / ba(d-2 * alpha) - 2
    subdom = brentq(Dalpha, alpha+.001, d - .000001 if d < 3 else 3 )
    return subdom


# theory curve (supp)

In [ ]:
rc('font', **{ "size":13,})

fig, ax = plt.subplots(1,2, figsize = (8, 3 ))
ax = ax.reshape(-1)

plt.sca(ax[0])
d = 1
alphas = np.linspace(d/2, min(d, 2), 200)[1:-1]
etas = np.array([geteta(alpha) for alpha in alphas])
kg = (1-etas)/(1-alphas)
kappa = (d - etas + alphas) 

plt.plot(alphas, kappa, "k-")

#plt.plot(alphas, (1+alphas)/2, "k--", lw=1)
plt.plot(alphas, alphas, "k--", lw=1)
plt.xlabel(r"$\alpha$", labelpad = 0)
plt.ylabel(r"$\chi$", labelpad = -0)
#plt.xticks([.5, 1], ["$1/2$", "$1$"])
plt.xlim(.5,1)
plt.ylim(.5,1)
#plt.yticks([.5, .75,  1], [r"$1/2$", r"$\frac34$", "$1$"])
plt.text(.52, 1.025, "(a) $d=1$")

plt.sca(ax[1])

d = 2
alphas = np.linspace(d/2, min(d, 2), 200)[1:-1]
etas = np.array([geteta(alpha) for alpha in alphas])
#kg = (1-etas)/(1-alphas)
kappa = (d - etas + alphas) 
plt.plot(alphas, kappa, "k-")

plt.plot(alphas, alphas, "k--", lw=1)
#plt.plot(alphas, (d+alphas)/2, "k--", lw=1)
#plt.xticks([1, 2], ["$1$", "$2$"])
plt.xlim(1,2)
plt.ylim(1,2)
#plt.yticks([1,   2], [r"$1$" ,"$2$"])
plt.xlabel(r"$\alpha$", labelpad = -0)
plt.text(1.04, 2.05, "(b) $d=2$")

plt.subplots_adjust(bottom=.2)

#plt.savefig("../Dropbox/Apps/Overleaf/galton-watson/paper1/kappa12.pdf")

In [ ]:
# deprecated
d = 3
rc('font', **{ "size":13,})
plt.figure(1, figsize = (4, 3))
alphas = np.linspace(d/2, min(d, 2), 200)[1:-1]
etas = np.array([geteta(alpha) for alpha in alphas])
#kg = (1-etas)/(1-alphas)
plt.plot(alphas, etas, "k-")

plt.plot(alphas, alphas, "k--", lw=1)
plt.xlabel(r"$\alpha$", labelpad = -10)
plt.ylabel(r"$\eta$", labelpad = -10)
plt.xticks([1.5, 2], ["$3/2$", "$2$"])
plt.xlim(1.5,2)
plt.ylim(1.5,3)
plt.yticks([1.5, 2, (1+17**.5)/2, 3], [r"$3/2$", "$2$", r"$\eta_{\mathrm{SR}}$", "$3$"])
plt.text(1.55, 2.8, "$d=3$")

plt.tight_layout()
plt.grid(1)
#plt.savefig("../Dropbox/Apps/Overleaf/galton-watson/paper1/eta3.pdf")

# gap distribution (main)

In [ ]:
from matplotlib import cm

In [ ]:
res = dict()

In [ ]:

d = 1


alpha = .6
if alpha > .5:
    eta = geteta(alpha)
    kg = (1-eta)/(1-alpha)
    print(kg)
else:
    kg = 1
N = 400
delta = .2
xmin = 1

endpoints = np.hstack([[0, xmin], np.exp(np.arange(1,N+1) * delta) * xmin ])
ls= endpoints[:-1]
rs = endpoints[1:]
mid = (endpoints[1:] + endpoints[:-1]) / 2
K = np.array([(abs(1/np.abs(x - rs) ** alpha - 1/np.abs(x-ls) ** alpha) 
    + abs(1/np.abs(x + rs) ** alpha - 1/np.abs(x  + ls) ** alpha)) for x in mid[1:]]) / alpha
K[np.arange(N), np.arange(N)+1]=0
K0 = np.sum(K, axis = 1)



m2s = .1 ** np.arange(10, 16)

nbs =  np.arange(20, 170, 5) 
bs = (np.exp(nbs * delta))
Ls = []
for m2 in m2s:
    F = np.zeros(N+1)
    ls = []
    for nb in nbs:
        if (nb,m2) in res: 
            F = res[nb, m2]
        else:
            F = np.zeros(N+1)
            F[:nb+1] = 1
            for j in range(2000):
                g = (K[nb:, :] @ F) 
                err = np.max((g - F[1+nb:] * (K0[nb:] + m2) - F[1+nb:]**2) * mid[1+nb:] ** alpha)
                if err < 1e-15: 
                    #print("converged at step", j, "error", err)
                    break
                F[nb+1:] = (- (m2  + K0[nb:]) + ((m2  + K0[nb:]) ** 2 +  g * 4) ** .5) / 2
                F[:nb+1] = 1
            res[nb, m2] = F[1:]
        ls.append(np.sum( np.diff(endpoints)[nb+1:] * F[nb+1:]))
    
    ls = np.diff(ls) / np.diff(bs)
    Ls.append(ls)

In [ ]:
rc('font', **{ "size":12,})

plt.figure(1, figsize = (4.3, 3.))
for m2, ls in zip(m2s,Ls):
    #plt.loglog(bs[1:] / m2 ** (-(1-alpha)/alpha), ls, ".")
    brs = bs[1:] / m2 ** (-(1-alpha)/alpha)
    plt.loglog(brs, ls  , ".", markersize = 3)
    if m2 == m2s[0]:
        xx= bs[1:] / m2 ** (-(1-alpha)/alpha)
        xx= xx[xx>1]
        plt.plot(xx, xx ** (-alpha/(alpha+1)) , "g--",lw = 1.5, )
xx= bs[1:] / m2 ** (-(1-alpha)/alpha)
xx= xx[xx<1]
plt.plot(xx, xx ** (-kg), "b--",lw = 1.5 )

#plt.plot(xx, xx ** (-1/(2*alpha))  , "r--",lw = 1 )
plt.xlabel(r"$g/g_c$", labelpad=0)
plt.ylabel(r"$\left< N_c(b = g) \right>_S / \sqrt{S}$", labelpad=0)
plt.tight_layout()
plt.subplots_adjust(left=.16, bottom=.16, top=.98, right=.98)

plt.annotate("bulk", xy=(1e-3,1e2),xytext=(1e-5, 1e0), arrowprops=dict(arrowstyle="-|>"),)

plt.annotate("outskirt", xy=(1e3, 7e-2),xytext=(1e0, 1e-3), arrowprops=dict(arrowstyle="-|>"),)


##########################
inset = plt.axes([.6, .6, .35, .35])
d = 1
alphas = np.linspace(d/2, min(d, 2), 200)[1:-1]
etas = np.array([geteta(alpha) for alpha in alphas])
kgs = (1-etas)/(1-alphas)

plt.plot(alphas, kgs, "b-",)
plt.plot(alphas, alphas / (1+alphas), "g-", )
alphas = np.linspace(0, .5, 200)
plt.plot(alphas, (alphas)/(1-alphas), "b-", label="small gap")
plt.plot(alphas, alphas / (1+alphas), "g-", label="large gap")
plt.xlabel(r"$\alpha$", labelpad = -1)
plt.xlim(0,1)
plt.ylim(0,1)
plt.ylabel(r"exponents")
plt.plot([alpha, alpha], [0, 1], "k--", lw=1)
plt.plot([alpha], [kg], "o", c="b" )
plt.plot([alpha], [alpha/(alpha+1)], "o", c="g" )
plt.xticks([0, 1/2, 1], ["$0$", "$1/2$", "$1$"])
plt.yticks([0 ,1/2, 1], ["$0$", r"$\frac12$", "$1$"])

##########################

#plt.savefig("/Users/xiangyucao/Dropbox/Apps/Overleaf/galton-watson/paper1/gap_main.pdf")

# traveling wave

In [ ]:
alpha = 2.

N = 400
delta = .1
xmin = 1

endpoints = np.hstack([[0, xmin], np.exp(np.arange(1,N+1) * delta) * xmin ])
ls= endpoints[:-1]
rs = endpoints[1:]
mid = (endpoints[1:] + endpoints[:-1]) / 2
K = np.array([(abs(1/np.abs(x - rs) ** alpha - 1/np.abs(x-ls) ** alpha) 
    + abs(1/np.abs(x + rs) ** alpha - 1/np.abs(x  + ls) ** alpha)) for x in mid[1:]]) / alpha
K[np.arange(N), np.arange(N)+1]=0
K0 = np.sum(K, axis = 1)

nbs = [20, 40, 60]
Ints = []
beta = 1
jmax = 4000
dt = .02
plt_waves = []
for nb in nbs:
    xs = mid

    F = np.zeros(N+1, dtype= "float128")
    F[:nb+1] = 1
    
    b = mid[nb+1]
    ints = []
    
    for j in range(jmax):
        g = (K[nb:, :] @ F) - K0[nb:] * F[nb + 1:] +  beta* F[nb + 1:]  - F[nb+1: ] ** 2
        #F[:nb+1] = 1
        F[nb+1:] += g * dt
        c = j / jmax / 2
        xi = mid[np.searchsorted(-F, -.5)] / np.exp(delta/2) #np.exp(j * dt * beta/ (alpha+1)) * b ** (1/(alpha+1))
        #ints.append(np.sum( np.diff(endpoints)[nb+1:] * F[nb+1:]))ints.append()
        #ints.append(np.sum( np.diff(endpoints)[nb+1:] * F[nb+1:]))
        ints.append(xi)
        if (j+1) in [1000, 2000, 4000]  and nb == nbs[0]:
             plt_waves.append(((j+1) * dt, xi, np.array(F)))
             #plt.loglog(xs[nb:]  / xi, F[nb:] , c=(1-c,0,c)  ) #
             #print((j+1) * dt)
    Ints.append(ints)

In [ ]:
             #print((j+1) * dt)
alpha = 2
rc('font', **{ "size":12,})
plt.figure(1,figsize = (4.5, 3))
nb = nbs[0]
for t, xi, F in plt_waves:
    c= (t-20)/60
    plt.loglog(xs[nb:] / xi  , F[nb:], c= (c,0,1-c) , label= r"$t = %g$" % t, alpha = .5) #
plt.legend(frameon = 0, bbox_to_anchor = (.72
                                          , .1), loc = 3,
           handlelength=1)

x1 = 10 ** np.linspace(-4,2,200)
y1 = 1/(1 + x1 ** (1+alpha))
#mid[np.searchsorted(-F, -.5)]
plt.plot(x1, 1/(1 + x1 ** (1+alpha)), "k--", lw=2)

plt.ylim(1e-4, 1.2)
plt.xlim(1e-1, 1e2)

plt.xlabel(r"$x / \xi(b)$", labelpad=-10)
plt.ylabel(r"$F$", labelpad=0)
#plt.annotate(r"$1/x^{\alpha+1}$", xy = (10, 1e-3), xytext = (10, 1e-4), 
#            arrowprops=dict(arrowstyle="-|>",
#                             linestyle= ls))
plt.annotate( r"$|x|^{-\alpha-1}$" , xy = (1e1, 1e-3), 
             xytext = (.5e1, 2e-4 ) ,arrowprops=dict(arrowstyle="-|>"))



plt.axes([.24, .25, .26, .45])
ts = (np.arange(len(ints))+1) * dt
cs= ["k", "b", "g", ]
for k,ints in enumerate(Ints):
    b = mid[nbs[k]+1]
    plt.plot(ts, np.array(ints)/b ** (1/(alpha+1)), c=cs[k], label="$%d$" % b)
plt.legend(frameon = 0, handlelength=.5, fontsize = 11, loc=2, title="$b$",
           labelspacing=.2, borderpad=0)
plt.yscale("log")
plt.xlabel(r"$t$", labelpad=-10)
plt.text(2, 2e7, r"$\xi(b) / b^{\frac{1}{\alpha+1}}$")
ts = np.linspace(10, 50, 10)
plt.plot(ts, np.exp(ts / (alpha+1)) / 2, 'r--')
plt.text(20, 1e2
         , r"$\exp(\frac{\beta-\gamma}{\alpha+1} t)$", color="red")
#plt.ylabel(, labelpad=-10)
plt.xticks([0, 50])
plt.xlim(-2,50)
plt.ylim(1,1e7)


plt.axes([.62, .62, .32, .32])
nb = nbs[0]
for t, xi, F in plt_waves:
    c= t/100
    plt.loglog(xs[nb:]   , F[nb:], c= (c,0,1-c), alpha = .7 ) #
plt.xlabel(r"$x$", labelpad=-10)
plt.ylabel(r"$F$", labelpad=-10)
#plt.ylim(1e-2, 2)
plt.xlim(1e2, 1e14)
plt.xticks([1e3, 1e12])
plt.yscale("linear")

plt.subplots_adjust(left = .15, bottom = .14, right = .96, top =.96)
#plt.savefig("../Dropbox/Apps/Overleaf/galton-watson/paper1/travel.pdf")

#  Scaling collapse of instanton solutions (SM)


In [ ]:
res = dict()

In [ ]:

d = 1


alpha = .75
if alpha > .5:
    eta = geteta(alpha)
    kg = (1-eta)/(1-alpha)
   # print(kg)
else:
    kg = 1
N = 440
delta = .2
xmin = 1

endpoints = np.hstack([[0, xmin], np.exp(np.arange(1,N+1) * delta) * xmin ])
ls= endpoints[:-1]
rs = endpoints[1:]
mid = (endpoints[1:] + endpoints[:-1]) / 2
K = np.array([(abs(1/np.abs(x - rs) ** alpha - 1/np.abs(x-ls) ** alpha) 
    + abs(1/np.abs(x + rs) ** alpha - 1/np.abs(x  + ls) ** alpha)) for x in mid[1:]]) / alpha
K[np.arange(N), np.arange(N)+1]=0
K0 = np.sum(K, axis = 1)


nbs =  np.arange(20, 110, 8) 
bs = (np.exp(nbs * delta))
Ls = []
m2 = 1e-19
F = np.zeros(N+1)
ls = []
for nb in nbs:
    if (nb,m2) in res: 
        F = res[nb, m2]
    else:
        F = np.zeros(N+1)
        F[:nb+1] = 1
        for j in range(2000):
            g = (K[nb:, :] @ F) 
            err = np.max((g - F[1+nb:] * (K0[nb:] + m2) - F[1+nb:]**2) * mid[1+nb:] ** alpha)
            if err < 1e-15: 
                #print("converged at step", j, "error", err)
                break
            F[nb+1:] = (- (m2  + K0[nb:]) + ((m2  + K0[nb:]) ** 2 +  g * 4) ** .5) / 2
            F[:nb+1] = 1
        res[nb, m2] = F[1:]
    #ls.append(np.sum( np.diff(endpoints)[nb+1:] * F[nb+1:]))

#ls = np.diff(ls) / np.diff(bs)
#Ls.append(ls)

In [ ]:
rc('font', **{ "size":13,})
plt.figure(1,figsize = (6.5, 4))
for nb in nbs[:2:-1]:
    F = res[nb, m2] 
    xs = mid[nb+1:]
    b = mid[nb]
    xb = b ** (1/(1-alpha))
    c = (nb - nbs[3]) / (nbs[-1] - nbs[3])
    label = None
    if nb == nbs[3]:
        label = r"$b/\epsilon = 6 \times 10^3$"
    if nb == nbs[-1]:
        label = r"$b/\epsilon = 2 \times 10^9$"
    plt.loglog(xs / xb, F[nb:] * xs ** alpha, c=(1-c,0,c), lw=1.5, label=label)
    print(xs[nb])
Calpha = ba(alpha) * ba(d-2 * alpha) * ba(d + alpha) / np.pi / 2
xs = np.linspace(1e0, 1e25)
plt.plot(xs, np.ones_like(xs) * Calpha, "--k", lw=2)
plt.annotate(r"$F_{\mathrm{sc}}(x) x^{\alpha}$", xy = (1e15, 1.5), xytext = (1e15, 10), 
            arrowprops=dict(arrowstyle="-|>"))
xs = np.linspace(1e-14, 1e-1)
plt.plot(xs, xs ** (-(1-alpha)/2) * 2 ** .5, "--k", lw=2)

plt.annotate(r"$F_{\mathrm{pl}}(x) x^{\alpha}$", xy = (1e-10, 1e2), xytext = (1e-10, 5), 
            arrowprops=dict(arrowstyle="-|>"))
plt.ylim(1e-2, 3e3)
plt.xlim(1e-15, 1e26)

plt.subplots_adjust(left = .13, bottom = .15, right = .98, top =.98)
plt.xlabel(r"$x/x_b = x / b^{1/(1-\alpha)}$")
plt.ylabel(r"$F(x) x^{\alpha}$")
plt.legend()
#plt.savefig("../Dropbox/Apps/Overleaf/galton-watson/paper1/collapse.pdf")

# short range numerics (SM)

In [ ]:
def getSR():
    xmin = 1
    alpha = 2.
    endpoints = np.hstack([[0, xmin], np.exp(np.arange(1,N+1) * delta) * xmin ])
    ls= endpoints[:-1]
    rs = endpoints[1:]
    mid = (endpoints[1:] + endpoints[:-1]) / 2
    K = np.array([(abs(1/np.abs(x - rs) ** alpha - 1/np.abs(x-ls) ** alpha) 
        + abs(1/np.abs(x + rs) ** alpha - 1/np.abs(x  + ls) ** alpha)) for x in mid[1:]]) / alpha
    #K = np.array([(abs(1/np.abs(x - rs) ** alpha - 1/np.abs(x-ls) ** alpha))
    #    for x in mid[1:]]) / alpha
    
    for j in range(N):
        K[j, j+3:]  = 0

        K[j, :j] = 0

    K[np.arange(N), np.arange(N)+1]=0
    K0 = np.sum(K, axis = 1)
    return K, K0

In [ ]:
delta = .1
alpha = 3.5
forward= False
N = 250
xmin = 1
endpoints = np.hstack([[0, xmin], np.exp(np.arange(1,N+1) * delta) * xmin ])
ls= endpoints[:-1]
rs = endpoints[1:]
mid = (endpoints[1:] + endpoints[:-1]) / 2
K = np.array([(abs(1/np.abs(x - rs) ** alpha - 1/np.abs(x-ls) ** alpha) 
    + abs(1/np.abs(x + rs) ** alpha - 1/np.abs(x  + ls) ** alpha)) for x in mid[1:]]) / alpha
    
K[np.arange(N), np.arange(N)+1]=0
K0 = np.sum(K, axis = 1)
p = 1
SR = 1
KSR, K0SR = getSR()
K += KSR 
K0 += K0SR 
################
ba = lambda h:2 * gamma(1-h) * np.sin(np.pi* h/ 2)
Calpha= ba(alpha) * ba(1-2 * alpha) * ba(alpha+1)  / np.pi / 2 
Dalpha = lambda gamma: ba(gamma) * ba(1-gamma-alpha) / ba(alpha) / ba(1-2 * alpha) - 2
if alpha < 1 and alpha > .5:
    subdom = brentq(Dalpha, .5001, .999)
    kappag = (1-subdom)/(1-alpha)
if alpha < .5:
    kappag = alpha / (1-alpha)

In [ ]:
res = dict()

In [ ]:
m2s = np.array([[x,x*1.5] for x in [1e-13, 1e-15, 1e-17, 1e-19]]).reshape(-1)
# [1e-22, 1.5e-22, 1e-20, 1.5e-20, 1e-18, 1.5e-18, 1e-16, 1.5e-16]
ba = lambda h:2 * gamma(1-h) * np.sin(np.pi* h/ 2)
Calpha= ba(alpha) * ba(1-2 * alpha) * ba(alpha+1)  / np.pi / 2 

nbs = np.arange(40, 150, 5) 
bs = (np.exp(nbs * delta))
Ls = []
for m2 in m2s:
    
    F = np.zeros(N+1)
    ls = []
    for nb in nbs:
        if (nb, m2) in res:
            F[1:] = res[nb, m2]
        else:
            print(m2, nb, end = " ")
            F = np.zeros(N+1)
            F[:nb+1] = 1
            for j in range(5000):
                g = (K[nb:, :] @ F) 
                err = np.max((g - F[1+nb:] * (K0[nb:] + m2) - p * F[1+nb:]**2) * mid[1+nb:] ** alpha)
                if err < 1e-20: 
                    #print("converged at step", j, "error", err)
                    break
                F[nb+1:] = (- (m2  + K0[nb:]) + ((m2  + K0[nb:]) ** 2 + p * g * 4) ** .5) / 2 / p
                F[:nb+1] = 1
            else:
                pass
                #print("warning conv", err)
            res[nb, m2] = F[1:]
        ls.append(np.sum( np.diff(endpoints)[nb+1:] * F[nb+1:]))
    ls = np.diff(ls) / np.diff(bs)
    Ls.append(ls)

In [ ]:
rc('font', **{ "size":13,})
plt.figure(1,figsize = (6.5, 4))
Ls1= ((np.diff(Ls, axis = 0).T / np.diff(m2s)).T)[::2]
for ls,m2 in zip(Ls1, m2s[::2]): 
    plt.loglog(bs[2:] / m2 ** (-(alpha-3)/2), np.abs(-ls[1:]) / m2 ** ((-4+alpha)/2)
               , ".", label=r"$S=10^{%d}$" % (int(-np.log(m2)*2 / np.log(10)+1)))
us = np.linspace(5, 1e3,5)
plt.plot(us, us ** - (alpha / (alpha+1)) / 5, "g--")
us = np.linspace(1e-3, .5, 5)
plt.plot(us, us ** 0 * .08,  "k--")
plt.annotate(r"$g^{-\alpha/(\alpha+1)}$", xy = (1e2, 5e-3), xytext = (1e2, 1e-2), 
            arrowprops=dict(arrowstyle="-|>"))
plt.xlabel(r"$g/g_c'$")
plt.ylabel(r"$\left< N_c(b=g) \right>_S / S^{1-\alpha/4}$")
plt.ylim(2e-4, 1e-1)
plt.tight_layout()
plt.legend()
#plt.savefig("/Users/xiangyucao/Dropbox/Apps/Overleaf/galton-watson/paper1/SR.pdf")